### **0. Set-up**

In [6]:
# Import libraries and utils
%run '../../utils.ipynb'

In [7]:
# Get api key
load_dotenv()
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')

# Set client
client = OpenAI()

In [8]:
# Load dataframe
en_simlex = pd.read_csv("../../../data/dataset/cleaned-en-simlex-999.csv")

# Select subset
# en_simlex = en_simlex.head(150)

# Convert to tuple
tuples_list = list(zip(en_simlex['word1'], en_simlex['word2']))

In [9]:
# Show results
tuples_list

[('old', 'new'),
 ('smart', 'intelligent'),
 ('hard', 'difficult'),
 ('happy', 'cheerful'),
 ('hard', 'easy'),
 ('fast', 'rapid'),
 ('happy', 'glad'),
 ('short', 'long'),
 ('stupid', 'dumb'),
 ('weird', 'strange'),
 ('wide', 'narrow'),
 ('bad', 'awful'),
 ('easy', 'difficult'),
 ('bad', 'terrible'),
 ('hard', 'simple'),
 ('smart', 'dumb'),
 ('insane', 'crazy'),
 ('happy', 'mad'),
 ('large', 'huge'),
 ('hard', 'tough'),
 ('new', 'fresh'),
 ('sharp', 'dull'),
 ('quick', 'rapid'),
 ('dumb', 'foolish'),
 ('wonderful', 'terrific'),
 ('strange', 'odd'),
 ('happy', 'angry'),
 ('narrow', 'broad'),
 ('simple', 'easy'),
 ('old', 'fresh'),
 ('apparent', 'obvious'),
 ('inexpensive', 'cheap'),
 ('nice', 'generous'),
 ('weird', 'normal'),
 ('weird', 'odd'),
 ('bad', 'immoral'),
 ('sad', 'funny'),
 ('wonderful', 'great'),
 ('guilty', 'ashamed'),
 ('beautiful', 'wonderful'),
 ('confident', 'sure'),
 ('dumb', 'dense'),
 ('large', 'big'),
 ('nice', 'cruel'),
 ('impatient', 'anxious'),
 ('big', 'broad'),

### **1. Define and Evaluate Parameters**

In [21]:
# Define prompt
prompt = ("Beoordeel de semantische gelijkenis van elk Engels woordpaar met een score op een schaal van 0 tot 10, "
          "waarbij 0 geen semantische gelijkenis vertegenwoordigt, en 10 perfecte semantische gelijkenis. "
          "Gebruik twee decimalen. Het antwoord moet strikt voldoen aan de structuur: "
          "[(woord1, woord2, <score>), (woord3, woord4, <score>),]. "
          " Geef geen extra uitleg of context.")

In [22]:
# Define model
model = "gpt-3.5-turbo-0125"

# Set sample size
sample_size = 15

# Delay between individual API calls
delay = 15.0

# Define number of sublists
n_sublists = 20

In [23]:
# Split list
chunks = split_into_n_lists(tuples_list, n_sublists)

# Count the number of lists
print(len(chunks))

20


In [24]:
# Print prompts for each chunk
print_prompts(chunks, prompt)

Beoordeel de semantische gelijkenis van elk Engels woordpaar met een score op een schaal van 0 tot 10, waarbij 0 geen semantische gelijkenis vertegenwoordigt, en 10 perfecte semantische gelijkenis. Gebruik twee decimalen. Het antwoord moet strikt voldoen aan de structuur: [(woord1, woord2, <score>), (woord3, woord4, <score>),].  Geef geen extra uitleg of context. --- ["('old', 'new'), ('smart', 'intelligent'), ('hard', 'difficult'), ('happy', 'cheerful'), ('hard', 'easy'), ('fast', 'rapid'), ('happy', 'glad'), ('short', 'long'), ('stupid', 'dumb'), ('weird', 'strange'), ('wide', 'narrow'), ('bad', 'awful'), ('easy', 'difficult'), ('bad', 'terrible'), ('hard', 'simple'), ('smart', 'dumb'), ('insane', 'crazy'), ('happy', 'mad'), ('large', 'huge'), ('hard', 'tough'), ('new', 'fresh'), ('sharp', 'dull'), ('quick', 'rapid'), ('dumb', 'foolish'), ('wonderful', 'terrific'), ('strange', 'odd'), ('happy', 'angry'), ('narrow', 'broad'), ('simple', 'easy'), ('old', 'fresh'), ('apparent', 'obvious

In [25]:
# Load encoding
encoding = tiktoken.get_encoding("cl100k_base")

# Count tokens per chunk
token_counts = count_tokens_with_tiktoken(chunks, prompt)

# Show results
print("Token counts for each formatted prompt:", token_counts)

Token counts for each formatted prompt: [476, 483, 464, 467, 458, 456, 471, 478, 477, 463, 463, 470, 473, 460, 466, 462, 453, 468, 474, 444]


### **2. Extract and Process Data**

In [26]:
# Get results from API
# response = get_responses(chunks, prompt, model, sample_size, delay)

Processing:   0%|          | 0/300 [00:00<?, ?chunk/s]

Processing: 100%|██████████| 300/300 [2:03:11<00:00, 24.64s/chunk]  

Total time taken: 7391.82 seconds


In [27]:
# Define filepath
file_path = '../../../data/response/en/gpt-3.5-turbo-0125/f6.json'

# Check if file already exists
if not os.path.exists(file_path):
    with open(file_path, 'w') as f:
        json.dump(response, f)
        print("File saved successfully.")
else:
    print("File already exists. JSON was not saved to prevent overwriting.")

File saved successfully.


In [29]:
# Process data into dictionary
data_dict = process_responses(response)

# Check for values higher/lower then sample size
higher_lower_samples = {key: value for key, value in data_dict.items() if len(value) < sample_size or len(value) > sample_size}

# Show results
print(higher_lower_samples)

{}


In [30]:
# Print duplicate word pairs
print_duplicate_word_pairs(en_simlex, data_dict)

Empty DataFrame
Columns: [Combined_Columns]
Index: []
Empty DataFrame
Columns: [Combined_Columns]
Index: []


In [31]:
# Convert dict to Pandas DataFrame
df = create_dataframe(data_dict)

# Show results
df

,word1,word2,similarity_score_1,similarity_score_2,similarity_score_3,similarity_score_4,similarity_score_5,similarity_score_6,similarity_score_7,similarity_score_8,similarity_score_9,similarity_score_10,similarity_score_11,similarity_score_12,similarity_score_13,similarity_score_14,similarity_score_15
0,old,new,0.0,0.19,1.00,0.07,0.02,0.04,0.01,2.50,0.01,1.0,0.01,0.15,0.04,0.01,0.04
1,smart,intelligent,10.0,0.83,1.00,0.88,0.94,0.95,9.50,7.50,8.00,1.0,9.50,0.95,1.00,9.00,0.91
2,hard,difficult,8.0,0.40,0.84,0.75,0.51,0.92,8.00,4.50,6.50,0.0,4.00,0.86,0.23,8.50,0.75
3,happy,cheerful,8.0,0.84,0.91,0.82,0.52,0.43,7.50,8.00,6.00,0.0,7.50,0.86,0.75,8.50,0.64
4,hard,easy,0.0,0.23,0.11,0.05,0.31,0.09,2.00,1.50,1.00,0.0,2.00,0.27,0.04,0.01,0.13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
994,join,acquire,4.0,4.00,4.00,0.12,0.31,2.00,0.40,0.62,2.75,6.0,4.00,0.49,0.45,6.00,0.08
995,send,attend,3.0,1.00,2.00,0.10,0.16,0.00,0.20,0.40,1.50,2.0,1.00,0.35,0.18,3.00,0.06
996,gather,attend,3.0,3.00,2.50,0.22,0.21,0.00,0.20,0.56,1.00,4.0,4.00,0.49,0.28,6.00,0.07
997,absorb,withdraw,1.0,1.00,2.00,0.08,0.28,0.00,0.20,0.44,1.00,1.0,1.00,0.37,0.25,2.00,0.05


In [32]:
# Count null values
count_null_values = df.isnull().sum()

# Show results
print("Null value counts per column:", count_null_values)

Null value counts per column: word1                  0
word2                  0
similarity_score_1     0
similarity_score_2     0
similarity_score_3     0
similarity_score_4     0
similarity_score_5     0
similarity_score_6     0
similarity_score_7     0
similarity_score_8     0
similarity_score_9     0
similarity_score_10    0
similarity_score_11    0
similarity_score_12    0
similarity_score_13    0
similarity_score_14    0
similarity_score_15    0
dtype: int64


In [33]:
# Check for rows with at least one null value
rows_with_null = df[df.isnull().any(axis=1)]

# Show results
rows_with_null

,word1,word2,similarity_score_1,similarity_score_2,similarity_score_3,similarity_score_4,similarity_score_5,similarity_score_6,similarity_score_7,similarity_score_8,similarity_score_9,similarity_score_10,similarity_score_11,similarity_score_12,similarity_score_13,similarity_score_14,similarity_score_15


In [34]:
# Show results
df

,word1,word2,similarity_score_1,similarity_score_2,similarity_score_3,similarity_score_4,similarity_score_5,similarity_score_6,similarity_score_7,similarity_score_8,similarity_score_9,similarity_score_10,similarity_score_11,similarity_score_12,similarity_score_13,similarity_score_14,similarity_score_15
0,old,new,0.0,0.19,1.00,0.07,0.02,0.04,0.01,2.50,0.01,1.0,0.01,0.15,0.04,0.01,0.04
1,smart,intelligent,10.0,0.83,1.00,0.88,0.94,0.95,9.50,7.50,8.00,1.0,9.50,0.95,1.00,9.00,0.91
2,hard,difficult,8.0,0.40,0.84,0.75,0.51,0.92,8.00,4.50,6.50,0.0,4.00,0.86,0.23,8.50,0.75
3,happy,cheerful,8.0,0.84,0.91,0.82,0.52,0.43,7.50,8.00,6.00,0.0,7.50,0.86,0.75,8.50,0.64
4,hard,easy,0.0,0.23,0.11,0.05,0.31,0.09,2.00,1.50,1.00,0.0,2.00,0.27,0.04,0.01,0.13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
994,join,acquire,4.0,4.00,4.00,0.12,0.31,2.00,0.40,0.62,2.75,6.0,4.00,0.49,0.45,6.00,0.08
995,send,attend,3.0,1.00,2.00,0.10,0.16,0.00,0.20,0.40,1.50,2.0,1.00,0.35,0.18,3.00,0.06
996,gather,attend,3.0,3.00,2.50,0.22,0.21,0.00,0.20,0.56,1.00,4.0,4.00,0.49,0.28,6.00,0.07
997,absorb,withdraw,1.0,1.00,2.00,0.08,0.28,0.00,0.20,0.44,1.00,1.0,1.00,0.37,0.25,2.00,0.05


In [35]:
# Define file_path
file_path = '../../../data/prompt/en/gpt-3.5-turbo-0125/f6.csv'

# Check if file already exists
if not os.path.exists(file_path):
    df.to_csv(file_path, index=False)
    print("File saved successfully.")
else:
    print("File already exists. Dataframe was not saved to prevent overwriting.")

File saved successfully.
